In [1]:
%pip install openai
import openai
import os
import google.generativeai as genai
import subprocess
import os
import pathlib
import textwrap
import subprocess
import os
import json
import re
import pathlib
import pandas as pd
import textwrap
# from  google.colab import drive
# from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown
from openai import OpenAI

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
#if using colab
# from google.colab import drive

# drive.mount('/content/drive')

In [69]:
with open('D:/LLM_GPT/APPCAIR/expDetails.json','r') as openfile:
  expDetails = json.load(openfile)

EXP_NAME = expDetails['ExpName']

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

GOOGLE_API_KEY=expDetails['GeminiAPIKEY']
#CLAUDE_API_KEY= expDetails['ClaudeAPIKEY']
GPT_API_KEY = expDetails['GPTAPIKEY']
os.environ['OPENAI_API_KEY'] = GPT_API_KEY

genai.configure(api_key=GOOGLE_API_KEY)

Gmodel = genai.GenerativeModel(expDetails['GeminiModel'])

GPTModel = expDetails['GPTModel']
openai.api_key = GPT_API_KEY
client = OpenAI()

RunCount = expDetails['RunCount']
RunCount = str(RunCount)

json_object = json.dumps(expDetails,indent = 4)
with open('D:/LLM_GPT/APPCAIR/expDetails.json','w') as openfile:
  openfile.write(json_object)

def fileReader(pathToFile):
  f = open(pathToFile,'r')
  s = f.read()
  f.close()
  return s

idea = fileReader(expDetails['ideaFilePath'])
context = fileReader(expDetails['contextFilePath'])
libraries = fileReader(expDetails['librariesPath'])
methodology =fileReader(expDetails['methodology'])
miscellaneous=fileReader(expDetails['miscellenous'])
idea = methodology+"\n\n"+ idea
Dataset = fileReader(expDetails['datasetLoadingCode'])
custom_dataset=expDetails['dataset']
packages=fileReader(expDetails['packages'])
model_methodology=fileReader(expDetails['model_methodology'])
model_refined_methodology=fileReader(expDetails['model_refined_methodology'])
model_refined_methodology=model_methodology+model_refined_methodology

In [6]:
packages

'!pip install -q numerapi umap-learn pandas pyarrow matplotlib lightgbm scikit-learn cloudpickle scipy==1.10.1 tabpfn'

In [7]:
import subprocess
packages = packages.replace('!', '')
subprocess.run(packages.split())

CompletedProcess(args=['pip', 'install', '-q', 'numerapi', 'umap-learn', 'pandas', 'pyarrow', 'matplotlib', 'lightgbm', 'scikit-learn', 'cloudpickle', 'scipy==1.10.1', 'tabpfn'], returncode=1)

In [70]:
train = pd.read_csv(custom_dataset)

In [35]:
SampleModelCode = fileReader(expDetails['sampleModelCode'])
Model = fileReader(expDetails['Model'])
filepath = expDetails['basecode_path']
with open(filepath, 'r') as f:
  Sample_code=f.read()
  Sample_code = Sample_code.strip()
# Remove single quotes if they are at the beginning and end of the string
  if Sample_code.startswith("'") and Sample_code.endswith("'"):
    Sample_code = Sample_code[1:-1]

In [36]:
exec(Sample_code)

25


In [37]:
with open('D:/LLM_GPT/APPCAIR/Docs/inputs/prompts/high_level_idea.txt','r') as f:
  Model_Idea = f.read()

In [68]:
completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
          {
                "role":"system",
                "content":f"You are a Machine Learning engineer. Your task is to Replace the existing models used in the given Python code using the {model_refined_methodology}\n\n"
            },
        {
            "role": "user",
            "content": f"Replace only the models used in \n\n {Sample_code} \n\nwith \n\n{Model} whose sample usage is given in \n\n {SampleModelCode}\n\n.Generate the entire code again.Data already loaded in 'train' variable"
        }
    ]
)
print(completion.choices[0].message.content)

In [ ]:
Code=completion.choices[0].message.content

In [28]:
completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {
            "role": "user",
            "content": f"Your task is to generate Python code  for :{model_refined_methodology}. Avoid assumptions and be creative in applying various ML techniques tailored to {Model_Idea}.Sample usage is given in:{SampleModelCode}.Don't blinldy copy paste the sample cod,use it for referebce only. Provide a comprehensive and detailed implementation without using placeholders or assumptions.Build up on \n\n{Code} \n\n"
        }
    ]
)

In [67]:
Code2=completion.choices[0].message.content

In [66]:
completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
          {
                "role":"system",
                "content":f"You are a Machine Learning engineer at an organization."
            },
        {
            "role": "user",
            "content": f"Your job is to make changes in the  code \n\n{Code2}\n\n.Your task is to  change the Python code as per \n\n{Model_Idea}\n\n such that it's consistent with {model_refined_methodology}.Don't change the strcuture of code at all.Just make the code compatible with the {Model_Idea} \n\n"
        },
        {
             "role": "user",
            "content": f"Don't assume anything and you are allowed to be creative by using different ML techniques to fit the :{Model_Idea}.Data already loaded in 'train' variable.NO BREVITY,PLACEHOLDES,ASSUMPTON.Give FULL IMPLEMENTATION"
        }
        
    ]
)
print(completion.choices[0].message.content)

In [47]:
Interimcode=completion.choices[0].message.content

In [65]:
import re
with open('completion[0].txt', 'w') as f:
    f.write(Interimcode)

# Print a confirmation message
print("Message content has been saved to 'completion[0].txt' file.")


# Print a confirmation message
print("Message content has been saved to 'completion[0].txt' file.")

# Read the content of the text file
with open('completion[0].txt', 'r') as file:
    text_content = file.read()

# Extract Python code blocks using regular expressions
python_code_blocks = re.findall(r'```python\s*([\s\S]+?)\s*```', text_content)

# Remove pip installs from Python code blocks
python_code_blocks_cleaned = []
for code_block in python_code_blocks:
    cleaned_block = re.sub(r'^!pip install .*\n', '', code_block, flags=re.MULTILINE)
    python_code_blocks_cleaned.append(cleaned_block)

# Write the cleaned Python code blocks to a new file
with open('extracted_final_code_cleaned.py', 'w') as file:
    for code_block in python_code_blocks_cleaned:
        file.write(code_block + '\n\n')


In [64]:
import traceback
import re
def debug_error(error_message, code):
    # Define the prompt with the error message
    prompt = f"Error message: {error_message}\n\nDebugging suggestions:"

    # Generate suggestions using GPT
    completion = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    #system="You are a data scientist.I have encountered an error.Please fix the error in a logical way and dont assume anything.Regenerate the entire code again.If you need any clarifications ask.Output the entire code again after error rectification""",
    messages=[       {
                "role": "system",
                "content": "You are a data scientist.I have encountered an error.Please fix the error in a logical way and don't assume anything.Regenerate the entire code again.Output the entire rectified code again after error rectification",
            },
        {
                "role": "user",
                "content": f"[Error]:{prompt}\n\n [code]:{code}.Please fix the error.Don't comment out anything.Generate the complete code"
            },
                     {
                "role": "user",
                "content": f"[Context]:{context}\n\n [Idea]:{model_refined_methodology} [libraries]:{libraries}\n\n"
            }
                     ]

)
    # Extract and return the debugging suggestions
    suggestions = completion.choices[0].message.content
    print("Debugging suggestions:")
    print(suggestions)
    python_code_match = re.search(r'```python(.+?)```', suggestions, re.DOTALL)
    python_code = python_code_match.group(1) if python_code_match else ""

    return python_code



with open('extracted_final_code_cleaned.py', 'r') as f:
  python_code = f.read()
# Extract Python code from the provided message (example)
  python_code_blocks = re.findall(r'```python(.+?)```', python_code)
  


    # Prepare the code for execution
  code = """{}
         """.format(python_code_blocks)

  error_message = ""
  while True:
      try:
        # Execute the code
        print(code)
        exec(code)
        # If execution is successful, break the loop
        print("Code executed successfully. No errors.")
        base_folder = expDetails['FolderName'].lstrip('/').replace('/', '\\')
        expname = expDetails['ExpName'] + RunCount
        filepath = os.path.join(base_folder, expname, 'Iterative.py')
        # Ensure the directory exists
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(filepath, 'w') as f:  # Changed 'r' to 'w' for writing
          f.write(code)
        print(f"Successfully wrote to {filepath}")
        expDetails['final'] = filepath
        with open('D:\LLM_GPT\APPCAIR\expDetails.json', 'w') as json_file:
          json.dump(expDetails, json_file, indent=2)

        break

      except (ValueError, Exception) as e:
            # Extract the error message
        error_message = str(e) + "\n" + traceback.format_exc()
        print("Error message:", error_message)

            # Debug the error message
        corrected_code = debug_error(error_message, code)
        print(error_message)

        if corrected_code:
          code = corrected_code
        else:
          print("Error correction failed. Exiting.")
          break


HUMAN INTERFERENCE
For instances of where manual feedback was required.
For example in the case of Conpred if the model was not ensembling or datascaling

In [59]:
base_folder = expDetails['FolderName'].lstrip('/').replace('/', '\\')
expname = expDetails['ExpName'] + RunCount
filepath = os.path.join(base_folder, expname, 'Iterative.py')
with open(filepath, 'r') as f:
  final = f.read()

In [ ]:
Prompt="""
Ensemble the models used in the code and do weighted average
"""
##this prompt can be changed as needed

In [63]:
completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
          {
                "role":"system",
                "content":f"You are a Machine Learning engineer. Your task is do as per the prompt \n\n"
            },
        {
            "role": "user",
            "content": f"Do:\n{Prompt} for {python_code}.Do not modify any segment any other segments of code.Generate the entire code again.Don't miss out on anything"
        }
    ],
    temperature=0
)
print(completion.choices[0].message.content)

In [61]:
expDetails['RunCount']=expDetails['RunCount']+1